## SQL Table Creation

#### 1. Connect to PostgreSQL:
 Let's start by creating the connection to your PostgreSQL instance using SQLAlchemy and psycopg2.

In [1]:
# Import the DBConnection class from scripts.db_connection
import os
os.chdir(r'c:\users\ermias.tadesse\10x\Centralize-Ethiopian-medical-business-data')  # Set the working directory to the project root
from scripts.db_connection import DBConnection
from sqlalchemy import MetaData, Table, Column, Integer, String, DateTime, ForeignKey
import logging

# Create a connection instance
db = DBConnection(dbname='Central_Medical_Warehouse', user='postgres', password='Ermi@123')

# Connect to the database
db.connect()

os.chdir(r'c:\Users\ermias.tadesse\10x\Centralize-Ethiopian-medical-business-data\logs')

# Set up logging
logging.basicConfig(filename='Load_Data_To_Database.log', level=logging.INFO,
                    format='%(asctime)s %(levelname)s:%(message)s')

logging.info(f'Successfully connected to The Postgrass database: {db.dbname}')

# Initialize metadata for creating tables
metadata = MetaData()

# Define your tables
channels_table = Table('channels', metadata,
    Column('channel_id', Integer, primary_key=True, autoincrement=True),
    Column('channel_name', String, unique=True, nullable=False)
)

senders_table = Table('senders', metadata,
    Column('sender_id', Integer, primary_key=True),
    Column('sender_name', String)  # Optional if you want to track sender names
)

messages_table = Table('messages', metadata,
    Column('message_id', Integer, primary_key=True),
    Column('message', String, nullable=False),
    Column('date', DateTime, nullable=False),
    Column('sender_id', Integer, ForeignKey('senders.sender_id'), nullable=False),
    Column('channel_id', Integer, ForeignKey('channels.channel_id'), nullable=False)
)

media_table = Table('media', metadata,
    Column('media_id', Integer, primary_key=True, autoincrement=True),
    Column('media_path', String, nullable=False),
    Column('message_id', Integer, ForeignKey('messages.message_id'), nullable=False)
)

# Create the tables in the PostgreSQL database
if db.engine:
    metadata.create_all(db.engine)
    print("Tables created successfully.")
    logging.info(f'Tables created successfully.')
else:
    print("Failed to create tables due to no connection.")
    logging.info(f'Failed to create tables due to no connection.')

# # Close the connection after operations are done
# db.close()


Successfully connected to the database!
Tables created successfully.


### Database Design 
Tables:
Channels Table:

channel_id: Primary Key, auto-incremented.
channel_name: Unique name of the Telegram channel.
Senders Table:

sender_id: Primary Key.
sender_name: (optional) - if the sender has an associated name or username.
Messages Table:

message_id: Primary Key.
message: The text of the message.
date: The date the message was posted.
sender_id: Foreign Key from the Senders table.
channel_id: Foreign Key from the Channels table.
Media Table:

media_id: Primary Key, auto-incremented.
media_path: Path to the media file.
message_id: Foreign Key from the Messages table.

Explanation of Tables:
channels: Stores unique channel names.
senders: Stores unique senders based on sender_id.
messages: Stores each message, linking it to a channel and sender.
media: Stores the media paths related to messages.

## Insert Data from scraped_data_cleaned.xlsx
Next, we’ll load the cleaned Excel data, split it into the respective tables, and insert it into the PostgreSQL database.

Load Excel Data and Prepare for Insertion

In [2]:
import pandas as pd
import logging
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import os

# Set up logging configuration
logging.basicConfig(level=logging.INFO)

# Change to your working directory
os.chdir(r'c:\Users\ermias.tadesse\10x\Centralize-Ethiopian-medical-business-data')

# Load cleaned data from Excel
scraped_data = pd.read_excel('data/cleaned/scraped_data_cleaned.xlsx')
logging.info('Read the cleaned data from Excel')
print("Read the cleaned data from Excel.")
# Create a session using the correct engine from DBConnection
# Assuming 'db.engine' is defined somewhere in your code
Session = sessionmaker(bind=db.engine)
session = Session()
logging.info('Created a session using the correct engine from DBConnection')

# Insert channels (bulk insert)
channels = [{'channel_name': channel} for channel in scraped_data['channel_name'].unique()]
session.execute(channels_table.insert().values(channels))
# Commit the session
session.commit()
logging.info('Bulk inserted channels')
logging.info("Data inserted channels successfully.")
print("channels Data inserted successfully.")
# Insert senders (bulk insert, cast to int)
senders = [{'sender_id': int(sender_id)} for sender_id in scraped_data['sender_id'].unique()]
session.execute(senders_table.insert().values(senders))
# Commit the session
session.commit()
logging.info('Bulk inserted senders')
logging.info("Data inserted senders successfully.")
print("senders Data inserted successfully.")
# Prepare messages and media for bulk insert
messages = []
media_entries = []

for index, row in scraped_data.iterrows():
    # Get channel_id for this message
    channel_id = session.execute(
        channels_table.select().where(channels_table.c.channel_name == row['channel_name'])
    ).fetchone()[0]
    
    # Prepare message entry
    messages.append({
        'message_id': int(row['message_id']),  # Cast to int
        'message': row['message'],
        'date': row['date'],
        'sender_id': int(row['sender_id']),  # Cast to int
        'channel_id': channel_id
    })

    # Prepare media entry if media path exists
    if pd.notnull(row['media_path']):
        media_entries.append({
            'media_path': row['media_path'],
            'message_id': int(row['message_id'])  # Cast to int
        })

# Proceed with insert if all message_ids are valid
if all(-2147483648 <= message['message_id'] <= 2147483647 for message in messages):
    session.execute(messages_table.insert().values(messages))
    logging.info('Bulk inserted messages')
else:
    logging.error('Bulk insert aborted due to invalid message_id values.')

# Bulk insert media if any entries exist
if media_entries:
    session.execute(media_table.insert().values(media_entries))
    logging.info('Bulk inserted media')

# Commit the session
session.commit()
logging.info("Data inserted successfully.")

# Close the session and connection
session.close()
logging.info("Session closed.")

print("Data inserted successfully.")



Read the cleaned data from Excel.
channels Data inserted successfully.
senders Data inserted successfully.
Data inserted successfully.


### Explanation:
Session Creation: The session is created using the engine from db.engine to manage transactions.


Data Insertion: Data from scraped_data_cleaned.xlsx is inserted into the normalized tables. The session.execute() method is used for the insertion of records into channels, senders, messages, and media tables.


Committing the Transaction: After all inserts, the session is committed to save the changes to the database.


Session and Connection Closing: Both the session and database connection are closed to ensure proper resource management.

  ### Senders data

In [3]:
# Define a query to fetch data from the xdr_data table
query = "SELECT * FROM senders;"

# Fetch data
data = db.fetch_data(query)

# Convert the data to a DataFrame
df = pd.DataFrame(data)

# Check if data is returned
if data is not None and not data.empty:
    # Display the DataFrame
    print(data.head())
else:
    print("No data available or data is empty.")

       sender_id sender_name
0 -1001102021238        None
1 -1001432982009        None
2 -1001447066276        None
3 -1001666492664        None


  ### messages data

In [4]:
# Define a query to fetch data from the xdr_data table
query = "SELECT * FROM messages;"

# Fetch data
data = db.fetch_data(query)

# Convert the data to a DataFrame
df = pd.DataFrame(data)

# Check if data is returned
if data is not None and not data.empty:
    # Display the DataFrame
    print(data.head())
else:
    print("No data available or data is empty.")

   message_id                                            message  \
0         864  https://youtu.be/5DBoEm-8kmA?si=LDLuEecNfULJVD...   
1         863  á‹¶áŠ­á‰°áˆ­áˆµ áŠ¢á‰µá‹®áŒµá‹« á‰  áŠ á‹²áˆµ ...   
2         862  áˆžá‰µ á‰ áˆµáŠ³áˆ­ \n\náˆˆáˆáŒ†á‰»á‰½áŠ• á‹¨...   
3         861  áŠ¨ HIV á‹¨á‰°áˆá‹ˆáˆ° áˆ°á‹ áŠ áŒ‹áŒ¥áˆŸá‰½...   
4         860  á‰ á‰…áˆ­á‰¥ áŒŠá‹œ á‰ áˆƒáŒˆáˆ«á‰½áŠ• áˆ‹á‹­ ...   

                 date      sender_id  channel_id  
0 2023-12-18 17:04:02 -1001102021238           1  
1 2023-11-03 16:14:39 -1001102021238           1  
2 2023-10-02 16:37:39 -1001102021238           1  
3 2023-09-16 07:54:32 -1001102021238           1  
4 2023-09-01 16:16:15 -1001102021238           1  


  ### media data

In [6]:
# Define a query to fetch data from the xdr_data table
query = "SELECT * FROM media;"

# Fetch data
data = db.fetch_data(query)

# Convert the data to a DataFrame
df = pd.DataFrame(data)

# Check if data is returned
if data is not None and not data.empty:
    # Display the DataFrame
    print(data.head())
else:
    print("No data available or data is empty.")

   media_id media_path  message_id
0         1   No Media         864
1         2   No Media         863
2         3   No Media         862
3         4   No Media         861
4         5   No Media         860


  ### channels data

In [5]:
# Define a query to fetch data from the xdr_data table
query = "SELECT * FROM channels;"

# Fetch data
data = db.fetch_data(query)

# Convert the data to a DataFrame
df = pd.DataFrame(data)

# Check if data is returned
if data is not None and not data.empty:
    # Display the DataFrame
    print(data.head())
else:
    print("No data available or data is empty.")

   channel_id       channel_name
0           1          DoctorsET
1           2              EAHCI
2           3          yetenaweg
3           4  lobelia4cosmetics
